In [1]:
#multi-agent testrun using pettingzoo
#still need to update the actionset
#still need to update reward function
#trial to see if it works

#update the actions so that it selects the frequency range of a particular radar first, and then picks a sub-band from inside it
#fix the frequency hopping of the radar so that it only transmits one frequency per radar per step.

#after finalizing actions and rewards, test on vanilla dqn and sstart implementing ddqn with per

#fixed the radar frquency issue, need to fix the states, obs, info etc tuple being returned by env to the dqn
#after that, train dqn
#finally move to ddqn with per

#created the ddqn with per, need to add multiple radars and finalize
#rn its trying to lock all jammers on to a single radar, which doesnt make sense and is the reason for no convergence

#GRAPH GEN DOES NOT WORK

#ddqn works well on test tho, generalises well to different hop lengths and radar freqs values
#the number of radars to jam is hardcoded as 3 in the env, need to change that to a variable

#FIND OUT HOW A DRQN WORKS AND IMPLEMENT DIAL
#MATHEMATICAL FORMULATION OF THE PROBLEM

In [2]:
import numpy as np
import math
import torch
import matplotlib.pyplot as plt
import gymnasium as gym
from gym import Env
from gym import spaces
import random
from IPython.display import clear_output
import os
import matplotlib
from collections import namedtuple, deque
from itertools import count
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from pettingzoo import ParallelEnv
import seaborn as sns

is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

#device setup
device = "cpu"

In [3]:
freq_agile_radar_versions = [[9 * math.pow(10, 9), 9.4* math.pow(10, 9)], [8 * math.pow(10, 9), 8.5* math.pow(10, 9)], [6 * math.pow(10, 9), 6.2* math.pow(10, 9)]]

hop_states = []

for i in range (0,3):
    radar_states = []
    interval = (freq_agile_radar_versions[i][1] - freq_agile_radar_versions[i][0])/10
    for j in range(0,10):
        radar_states.append(int(freq_agile_radar_versions[i][0]+ interval * j))
    hop_states.append(radar_states)

print(hop_states)


[[9000000000, 9040000000, 9080000000, 9120000000, 9160000000, 9200000000, 9240000000, 9280000000, 9320000000, 9360000000], [8000000000, 8050000000, 8100000000, 8150000000, 8200000000, 8250000000, 8300000000, 8350000000, 8400000000, 8450000000], [6000000000, 6020000000, 6040000000, 6060000000, 6080000000, 6100000000, 6120000000, 6140000000, 6160000000, 6180000000]]


In [4]:
jamming_bandwidths = [10e6, 20e6, 30e6]

In [5]:
class ParallelRadarJammingEnv(ParallelEnv):
    def __init__(self, hop_states, jamming_bandwidths, max_hop_length=8):
        super().__init__()

        self.hop_states = hop_states
        self.max_hop_length = max_hop_length

        #params for each hop state list
        self.low = [hop[0] for hop in self.hop_states]
        self.interval = [hop[1] - hop[0] for hop in self.hop_states]
        self.n_frequencies = [len(hop) for hop in self.hop_states]

        #agents
        self.agents = ["jammer_0", "jammer_1", "jammer_2", "jammer_3", "jammer_4", "jammer_5"]
        self.possible_agents = self.agents[:]
        self.agent_name_mapping = {agent: i for i, agent in enumerate(self.agents)}

        #action space
        self.jamming_bandwidths = jamming_bandwidths    
        self.n_bandwidths = len(self.jamming_bandwidths)
        self.action_space = {agent: spaces.Discrete(self.max_hop_length * self.n_bandwidths) for agent in self.agents}

        #observation space
        self.observation_spaces = {
            agent: spaces.Box(low=0, high=1, shape=(sum(self.n_frequencies),), dtype=np.float32) for agent in self.agents
        }

        self.hopping_patterns = None
        self.current_frequencies = None
        self.current_steps = None
        self.cumulative_rewards = None
        self.net_agent_rewards = None
        self.terminations = None
        self.rewards = None
        self.info = None

        
        #jamming fractions
        self.jamming_fractions = None
        self.episode_jamming_success = np.zeros((len(self.hop_states), 200))

    def index_to_action(self, index):
        #action index to freq + bandwidth
        frequency_index = index // self.n_bandwidths
        bandwidth_index = index % self.n_bandwidths
        hop_index = frequency_index % len(self.hop_states)
        frequency = int(self.low[hop_index] + (frequency_index // len(self.hop_states)) * self.interval[hop_index])
        bandwidth = self.jamming_bandwidths[bandwidth_index]
        return frequency, bandwidth

    def generate_hopping_pattern(self):
        return [np.random.choice(hop, self.max_hop_length, replace=False) for hop in self.hop_states]

    def reset(self):
        self.hopping_patterns = self.generate_hopping_pattern()

        # select two unique, non-overlapping frequencies from each pattern
        self.radar_frequencies = [
            [pattern[0], pattern[1]] for pattern in self.hopping_patterns
        ]

        self.current_steps = {agent: 0 for agent in self.agents}
        self.cumulative_rewards = {agent: 0 for agent in self.agents}
        self.terminations = {agent: False for agent in self.agents}
        self.rewards = {agent: 0 for agent in self.agents}
        self.info = {agent: {} for agent in self.agents}

        observations = {agent: self.observe(agent) for agent in self.agents}
        return observations

    def observe(self, agent):
        # observation is a one-hot encoded vector representing the radar frequencies across all hop_states
        observation = np.zeros(sum(self.n_frequencies))
        offset = 0
        for i, frequencies in enumerate(self.hop_states):
            for radar_freq in self.radar_frequencies[i]:  # both frequencies are now represented
                freq_index = np.where(np.array(frequencies) == radar_freq)[0][0]
                observation[offset + freq_index] = 1
            offset += len(frequencies)
        return observation


    def step(self, actions):
        total_jammed_frequencies = 0
        self.net_agent_rewards = 0

        #jamming success rate
        jammed_array = np.zeros_like(self.radar_frequencies)

        for agent, action in actions.items():
            frequency, bandwidth = self.index_to_action(action)
            lower_bound = frequency - bandwidth
            upper_bound = frequency + bandwidth

            jammed = False

            for radar_frequencies in self.radar_frequencies:
                for radar_frequency in radar_frequencies:
                    if lower_bound <= radar_frequency <= upper_bound:
                        jammed = True
                        reward = 30 * ([np.where(np.array(hop) == frequency)[0][0] for hop in self.hop_states if frequency in hop][0] + 1)
                        total_jammed_frequencies += 1
                        #updating jammed array
                        jammed_array[self.radar_frequencies.index(radar_frequencies)][radar_frequencies.index(radar_frequency)] = 1


            if jammed == False:
                reward = -50
                
            for i in range(len(jammed_array)):
                if np.all(jammed_array[i] == 1):
                    self.episode_jamming_success[i][self.current_steps[agent]] = 1

            # if jammed the reward is 30 * (the element of hop_states that the frequency is found in + 1)

            hop_index = np.argmin([abs(frequency - radar[0]) for radar in self.radar_frequencies])

            if 2 * (bandwidth / self.interval[hop_index]) > 5:
                reward -= 10

            jam_threshold = len(self.hop_states[0]) / len(self.agents)
            if total_jammed_frequencies > jam_threshold:
                reward += 5
            else:
                reward -= 5
            
            self.current_steps[agent] += 1
            self.cumulative_rewards[agent] += reward
            self.net_agent_rewards += reward

            # Termination condition after 200 steps
            if self.current_steps[agent] >= 200:
                self.terminations[agent] = True

            self.rewards[agent] = reward

        # Update radar frequencies for the next step with no overlap between two frequencies
        self.radar_frequencies = [
            [
                pattern[self.current_steps[agent] % self.max_hop_length],  # first frequency
                pattern[(self.current_steps[agent] + 2) % self.max_hop_length]  # second frequency ensuring no overlap
            ]
            for pattern in self.hopping_patterns
        ]
        if all(self.terminations.values()):
            self.jamming_fractions = np.sum(self.episode_jamming_success, axis=1) / 200
        
        self.episode_jamming_success = np.zeros_like(self.episode_jamming_success)

        # Return observations, rewards, done status, and info for each agent
        observations = {agent: self.observe(agent) for agent in self.agents}
        rewards = {agent: self.rewards[agent] for agent in self.agents}
        done = {agent: self.terminations[agent] for agent in self.agents}
        info = {agent: self.info[agent] for agent in self.agents}

        return observations, rewards, done, info

    def render(self):
        for agent in self.agents:
            print(f"Agent {agent}, Step: {self.current_steps[agent]}, Radar Frequencies: {self.radar_frequencies}")


In [6]:
env = ParallelRadarJammingEnv(hop_states, jamming_bandwidths)

observations = env.reset()
done = {agent: False for agent in env.agents}
cumulative_reward = {agent: 0 for agent in env.agents}

while not all(done.values()):

    actions = {agent: env.action_space[agent].sample() for agent in env.agents}
    observations, rewards, done, infos = env.step(actions)

    for agent in env.agents:
        cumulative_reward[agent] += rewards[agent]

    env.render()
    print(f"Actions: {actions}, Rewards: {rewards}, Cumulative Rewards: {cumulative_reward}")
    print('\n')


Agent jammer_0, Step: 1, Radar Frequencies: [[9120000000, 9240000000], [8050000000, 8000000000], [6060000000, 6180000000]]
Agent jammer_1, Step: 1, Radar Frequencies: [[9120000000, 9240000000], [8050000000, 8000000000], [6060000000, 6180000000]]
Agent jammer_2, Step: 1, Radar Frequencies: [[9120000000, 9240000000], [8050000000, 8000000000], [6060000000, 6180000000]]
Agent jammer_3, Step: 1, Radar Frequencies: [[9120000000, 9240000000], [8050000000, 8000000000], [6060000000, 6180000000]]
Agent jammer_4, Step: 1, Radar Frequencies: [[9120000000, 9240000000], [8050000000, 8000000000], [6060000000, 6180000000]]
Agent jammer_5, Step: 1, Radar Frequencies: [[9120000000, 9240000000], [8050000000, 8000000000], [6060000000, 6180000000]]
Actions: {'jammer_0': 20, 'jammer_1': 5, 'jammer_2': 23, 'jammer_3': 1, 'jammer_4': 7, 'jammer_5': 20}, Rewards: {'jammer_0': -55, 'jammer_1': -55, 'jammer_2': -55, 'jammer_3': -55, 'jammer_4': -55, 'jammer_5': -55}, Cumulative Rewards: {'jammer_0': -55, 'jammer

In [7]:
#setting up replay memory

Transition = namedtuple('Transition',
('state', 'action', 'next_state', 'reward')
)

In [8]:
class SequentialReplayBuffer:
    def __init__(self, maxlen=16000):
        self.buffer = deque(maxlen=maxlen)

    def append(self, trajectory):
        """Add a sequence of transitions to the buffer."""
        self.buffer.append(trajectory)

    def sample(self, batch_size, sequence_length):
        """Sample sequences of transitions."""
        if len(self.buffer) < batch_size:
            return []
        indices = np.random.choice(len(self.buffer), batch_size, replace=False)
        sampled = [self.buffer[idx] for idx in indices]
        return [self._pad_sequence(trajectory, sequence_length) for trajectory in sampled]

    @staticmethod
    def _pad_sequence(sequence, sequence_length):
        """Pad or truncate sequences to match `sequence_length`."""
        if len(sequence) >= sequence_length:
            return sequence[-sequence_length:]
        return [sequence[0]] * (sequence_length - len(sequence)) + sequence

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from collections import namedtuple, deque
from copy import deepcopy

# Define the Transition tuple
Transition = namedtuple('Transition', ('state', 'action', 'next_state', 'reward'))

class DRQNWithCommunication(nn.Module):
    def __init__(self, input_dim, hidden_dim, action_dim, message_dim=32):
        super(DRQNWithCommunication, self).__init__()
        self.hidden_dim = hidden_dim
        self.message_dim = message_dim

        # Observation encoder
        self.obs_encoder = nn.Linear(input_dim, hidden_dim)

        # RNN for temporal processing
        self.rnn = nn.LSTM(hidden_dim + message_dim, hidden_dim, batch_first=True)

        # Action-value output
        self.q_net = nn.Linear(hidden_dim, action_dim)

        # Message output
        self.message_net = nn.Linear(hidden_dim, message_dim)

    def forward(self, obs, hidden_state, message=None):
        """
        Forward pass for DRQN with communication.
        Args:
            obs: Observations (batch_size x input_dim).
            hidden_state: Hidden state for the RNN.
            message: Incoming message (batch_size x message_dim).
        """
        batch_size = obs.size(0)

        # Encode observation
        encoded_obs = self.obs_encoder(obs)  # Shape: (batch_size, hidden_dim)

        # Use default message if none provided
        if message is None:
            message = torch.zeros(batch_size, self.message_dim, device=obs.device)  # Shape: (batch_size, message_dim)
        if message.dim() == 3:
            message = message.squeeze(1)

        # Concatenate observation encoding and message
        rnn_input = torch.cat([encoded_obs, message], dim=-1)  # Shape: (batch_size, hidden_dim + message_dim)

        # RNN processing
        rnn_output, hidden_state = self.rnn(rnn_input.unsqueeze(1), hidden_state)  # RNN expects 3D input: (batch_size, seq_len, input_dim)
        rnn_output = rnn_output.squeeze(1)  # Shape: (batch_size, hidden_dim)

        # Compute Q-values and outgoing message
        q_values = self.q_net(rnn_output)  # Shape: (batch_size, action_dim)
        outgoing_message = self.message_net(rnn_output)  # Shape: (batch_size, message_dim)

        return q_values, outgoing_message, hidden_state

    def init_hidden(self, batch_size, device):
        """Initialize the hidden state for the RNN."""
        return (torch.zeros(1, batch_size, self.hidden_dim, device=device),
                torch.zeros(1, batch_size, self.hidden_dim, device=device))


In [10]:
class CommunicationAgent:
    def __init__(self, input_dim, hidden_dim, action_dim, message_dim, lr=1e-4, gamma=0.90, device="cpu"):
        self.device = device
        self.gamma = gamma

        # DRQN with communication
        self.model = DRQNWithCommunication(input_dim, hidden_dim, action_dim, message_dim).to(device)
        self.target_model = deepcopy(self.model)

        # Optimizer
        self.optimizer = optim.Adam(self.model.parameters(), lr=lr)

        # Replay buffer
        self.memory = deque(maxlen=10000)

    def act(self, state, hidden_state, message, epsilon):
        """
        Select an action using epsilon-greedy policy.
        Args:
        state: The current observation (state) for the agent.
        hidden_state: The hidden state for the RNN.
        message: The incoming communication message (None if no message).
        epsilon: Probability of taking a random action.
        Returns:
        action: Selected action.
        outgoing_message: The message generated for communication.
        """
        # Use default message if none is provided
        if message is None:
            message = torch.zeros(self.model.message_dim, device=self.device)  # Correctly initialize default message

        # Epsilon-greedy action selection
        if np.random.rand() < epsilon:
            return np.random.randint(0, self.model.q_net.out_features), np.zeros(self.model.message_dim)

        # Prepare inputs
        state_tensor = torch.FloatTensor(state).to(self.device).unsqueeze(0)
        message_tensor = torch.FloatTensor(message).to(self.device).unsqueeze(0)

        # Get Q-values and outgoing message
        q_values, outgoing_message, _ = self.model(state_tensor, hidden_state, message_tensor)

        # Select action
        action = q_values.argmax().item()

        return action, outgoing_message.detach().cpu().numpy()



    def remember(self, *args):
        """Store experiences in the replay buffer."""
        self.memory.append(Transition(*args))

    def replay(self, batch_size, sequence_length=10):
        if len(self.memory) < batch_size:
            return

        # Sample sequences of transitions
        batch = random.sample(self.memory, batch_size)
        transitions = Transition(*zip(*batch))

        # Convert the sampled transitions into tensors
        states = torch.FloatTensor(transitions.state).to(self.device)
        actions = torch.LongTensor(transitions.action).to(self.device)
        rewards = torch.FloatTensor(transitions.reward).to(self.device)
        next_states = torch.FloatTensor(transitions.next_state).to(self.device)

        # Compute target Q-values
        with torch.no_grad():
            target_q_values, _, _ = self.target_model(next_states, self.model.init_hidden(batch_size, self.device))
            max_next_q_values = target_q_values.max(dim=1)[0]
            targets = rewards + self.gamma * max_next_q_values

        # Compute predicted Q-values
        predicted_q_values, _, _ = self.model(states, self.model.init_hidden(batch_size, self.device))
        predicted_q_values = predicted_q_values.gather(1, actions.unsqueeze(1)).squeeze(1)

        # Loss and optimization
        loss = nn.MSELoss()(predicted_q_values, targets)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()


    def update_target_model(self):
        """Update target model weights."""
        self.target_model.load_state_dict(self.model.state_dict())


In [ ]:
# Example usage with ParallelRadarJammingEnv
env = ParallelRadarJammingEnv(hop_states, jamming_bandwidths)

# Define agents
agents = {name: CommunicationAgent(input_dim=sum(env.n_frequencies), hidden_dim=128, action_dim=env.action_space[name].n, message_dim=16, device=device) for name in env.agents}

messages = {agent: np.zeros(agents[agent].model.message_dim) for agent in env.agents}
hidden_states = {agent: agents[agent].model.init_hidden(1, device="cpu") for agent in env.agents}
epsilon = 1.0

episodes = 8000

for episode in range(1, episodes + 1):
    state = env.reset()
    total_reward = 0
    done = {agent: False for agent in env.agents}

    while not all(done.values()):
        actions = {}
        for agent in env.agents:
            actions[agent], messages[agent] = agents[agent].act(
                state[agent], hidden_states[agent], messages[agent], epsilon
            )

        next_state, rewards, done, _ = env.step(actions)
        total_reward += sum(rewards.values())

        # Store transitions for each agent
        for agent in env.agents:
            agents[agent].remember(state[agent], actions[agent], next_state[agent], rewards[agent])

        state = next_state

    epsilon = max(0.10, epsilon * 0.995)  # Decay exploration rate

    # Training
    for agent in agents:
        agents[agent].replay(batch_size=16)
        if episode % 20 == 0:
            agents[agent].update_target_model()

    print(f"Episode {episode}, Total Reward: {total_reward}")

C:\Users\Abhim\AppData\Local\Temp\ipykernel_37356\417115417.py:63: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:248.)
  states = torch.FloatTensor(transitions.state).to(self.device)


Episode 1, Total Reward: -29450
Episode 2, Total Reward: -40320
Episode 3, Total Reward: -40680
Episode 4, Total Reward: -39610
Episode 5, Total Reward: -23780
Episode 6, Total Reward: -33910
Episode 7, Total Reward: -35370
Episode 8, Total Reward: -37490
Episode 9, Total Reward: -44500
Episode 10, Total Reward: -34440
Episode 11, Total Reward: -27510
Episode 12, Total Reward: -29220
Episode 13, Total Reward: -30680
Episode 14, Total Reward: -41290
Episode 15, Total Reward: -30130
Episode 16, Total Reward: -30170
Episode 17, Total Reward: -31320
Episode 18, Total Reward: -36510
Episode 19, Total Reward: -31220
Episode 20, Total Reward: -34830
Episode 21, Total Reward: -27180
Episode 22, Total Reward: -32380
Episode 23, Total Reward: -29220
Episode 24, Total Reward: -39380
Episode 25, Total Reward: -30510
Episode 26, Total Reward: -28470
Episode 27, Total Reward: -25010
Episode 28, Total Reward: -36450
Episode 29, Total Reward: -44100
Episode 30, Total Reward: -23770
Episode 31, Total R